In [1]:
import json
from pathlib import Path
import pandas as pd

### Eye Closure Signal Files

file pattern: {subject_id}{session_id}{session_type}.json

### general
* 9 subjects
* 3 session with each subject except subject 9 and 14
* all 3 session types for each except subject 9 and 14 only type 'b'
* index 0 corresponds to the first frame of the experiment (?)
* some entries can be none if no eye is detected (?)


In [2]:
files = [str(p) for p in Path("potsdam_aeye_112020/").iterdir()]
for name in sorted(files):
    print(name.split("/")[1])


001_1_a.json
001_2_s.json
001_3_b.json
002_1_b.json
002_2_a.json
002_3_s.json
003_1_b.json
003_2_s.json
003_3_a.json
004_1_s.json
004_2_a.json
004_3_b.json
005_1_s.json
005_2_b.json
005_3_a.json
008_1_b.json
008_2_a.json
008_3_s.json
009_1_b.json
011_1_s.json
011_2_b.json
011_3_a.json
014_1_b.json


### Take a look at the data

### one sample:
* eye_closure for left and right image + combined 
* eye_state for left and right image + combined


In [3]:
with open(files[0]) as fp:
    data = json.loads(fp.read())
print(data[0])


{'eye_closure': {'combined': 0.39254617901185185, 'left_image': 0.3144874699612785, 'right_image': 0.4706048880624252}, 'eye_state': {'combined': 0, 'left_image': 0, 'right_image': 0}, 'index': 0}


In [4]:
df_eye_closure = pd.DataFrame([item["eye_closure"] for item in data])
df_eye_state = pd.DataFrame([item["eye_state"] for item in data], dtype="category")


In [5]:
df_eye_closure.describe()

,combined,left_image,right_image
count,148169.000000,148082.000000,148106.000000
mean,0.130700,0.151940,0.108870
std,0.112435,0.115879,0.111870
min,-0.338873,-0.436677,-0.310959
25%,0.077563,0.093926,0.055606
50%,0.103476,0.119726,0.087553
75%,0.139770,0.164167,0.124435
max,1.000000,1.000000,1.000000


In [6]:
print(df_eye_state["combined"].unique())
df_eye_state.describe()

[0.0, 1.0, 2.0, NaN, 5.0, 4.0]
Categories (5, float64): [0.0, 1.0, 2.0, 5.0, 4.0]


,combined,left_image,right_image
count,148200.0,148200.0,148200.0
unique,5.0,5.0,6.0
top,0.0,0.0,0.0
freq,146176.0,146015.0,146471.0


In [7]:
df_closure_and_state = df_eye_closure.join(df_eye_state, rsuffix="_eye_state", lsuffix="_eye_closure")

### Questions:
- What does it mean if the eye_closure value is negative? -- shouldnt be the case -> set to zero
- What does the 'eye_state' represent? --> documentation (should be available in moodle soon)
- What are session types: a,b,s ? (alcohol, baseline, sleep deprived)
- Why are some entries None ? -> no eyes in the image


### Further steps

- labels are not given for all frames -> how to interpolate labels?
    * split frames between end of first block and beginning of second block into the labels which are closer
    * linear interpolation
- calculate features like blink properties and time of closed pair of eyes
    - train linear classifier on these features
- train neural network on eye closure signal and compare to linear classifier

In [8]:
from functools import reduce

#combined_nans = set(df_eye_closure.loc[df_eye_closure["combined"].isna()].index)

def get_nan_intersections(df: pd.DataFrame):
    nan_indices = dict()
    for column in df.columns:
        nan_indices[column] = set(df.loc[df[column].isna()].index)
    
    for column in df.columns:
        nan_indices_copy = nan_indices.copy()
        col_indices = set(nan_indices_copy.pop(column))
        other_indices = reduce(lambda x,y: set(x) | set(y), nan_indices_copy.values())
        unique_nans = col_indices - other_indices
        print(f"Column {column} has {len(unique_nans)} nans that appear in no other column.")
    
        
        
get_nan_intersections(df_closure_and_state)

Column combined_eye_closure has 0 nans that appear in no other column.
Column left_image_eye_closure has 87 nans that appear in no other column.
Column right_image_eye_closure has 63 nans that appear in no other column.
Column combined_eye_state has 0 nans that appear in no other column.
Column left_image_eye_state has 0 nans that appear in no other column.
Column right_image_eye_state has 0 nans that appear in no other column.


In [9]:
reduce(lambda x,y: set(x) | set(y), [[1], [2]])

{1, 2}

## Target data 

In [10]:
session_identifier = "001_1_a"
files = [str(p) for p in Path("sleep_alc_labels/").iterdir()]
    
session_files = [file for file in files if session_identifier in file]
print("\n".join(session_files))

sleep_alc_labels/001_1_a_alcohol_consumptions.csv
sleep_alc_labels/001_1_a_karolinska.csv
sleep_alc_labels/001_1_a_pvt_scores.csv
sleep_alc_labels/001_1_a_pvt_reaction_times.csv
sleep_alc_labels/001_1_a_alcohol_measurements.csv


In [11]:
df_pvt_reaction_times = pd.read_csv("sleep_alc_labels/001_1_a_pvt_reaction_times.csv")
df_alcohol_measurement = pd.read_csv("sleep_alc_labels/001_1_a_alcohol_measurements.csv")
df_karolinka = pd.read_csv("sleep_alc_labels/001_1_a_karolinska.csv")
df_alcohol_consumptions = pd.read_csv("sleep_alc_labels/001_1_a_alcohol_consumptions.csv")
df_pvt_scores = pd.read_csv("sleep_alc_labels/001_1_a_pvt_scores.csv")

In [12]:
df_alcohol_consumptions.head(2)

,frame_begin,frame_end
0,13397,14760
1,38835,40104


In [18]:
df_alcohol_measurement.head(2)

,frame_begin,frame_end,promille
0,9480,13377,0.00
1,37520,38814,0.08


In [14]:
df_karolinka.head()

,frame_begin,frame_end,response_karolinska
0,8976,9457,6
1,37179,37497,4
2,46970,47153,4
3,49657,49757,4
4,55587,55757,3


In [15]:
df_pvt_reaction_times

,frame_begin,frame_end,reaction_time,block_id
0,17639,17767,266.39,1
1,17767,18107,278.11,1
2,18107,18265,259.19,1
3,18265,18514,266.58,1
4,18514,18703,271.66,1
...,...,...,...,...
239,166823,167105,360.59,3
240,167105,167326,315.04,3
241,167326,167667,331.77,3
242,167667,167830,403.87,3


In [16]:
df_pvt_scores

,frame_begin,frame_end,pvt_n_lapses_500ms,pvt_n_lapses_60s,pvt_median_rt,pvt_mean_rt,pvt_mean_log_rt,pvt_mean_slowest_10_percent_rt,pvt_mean_fastest_10_percent_rt
0,14762,35638,1,0,304.860,313.137590,5.738631,398.075556,266.249000
1,83408,102683,0,0,304.095,311.455976,5.734481,396.156667,265.771000
2,147900,168111,2,0,310.120,333.161266,5.778298,543.625000,246.265556
